In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('./test_score.ipynb'))))

In [2]:
import numpy as np
import pandas as pd

from ylearn.estimator_model._generalized_forest.tree._grf_tree import GrfTree
from ylearn.exp_dataset.exp_data import sq_data
from ylearn.utils._common import to_df
from ylearn.estimator_model._naive_forest.utils import grad, grad_coef, inverse_grad

from numpy.linalg import lstsq, inv

In [3]:
from copy import deepcopy


n = 2000
d = 10     
n_x = 1
y, x, v = sq_data(n, d, n_x)

true_te = lambda X: np.hstack([X[:, [0]]**2 + 1, np.ones((X.shape[0], n_x - 1))])
w = deepcopy(v)
v_test = v[:min(100, n)].copy()
v_test[:, 0] = np.linspace(np.percentile(v[:, 0], 1), np.percentile(v[:, 0], 99), min(100, n))

In [4]:
grftree = GrfTree(max_depth=10, max_leaf_nodes=10, min_samples_leaf=2)
grftree._fit_with_array(x=x, y=y, w=None, v=v)

c_init criterion
init splitter
build tree
start building tree 
start init tree with init_capacity=19entering nogil loop
start adding split node
reset criterion
reset criterion
reset criterion
reset criterion
reset criterion
reset criterion
reset criterion
reset criterion
reset criterion


GrfTree(max_depth=10, min_samples_leaf=2, max_leaf_nodes=10, categories=None)